In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen3:4b-instruct-2507-q4_K_M", temperature=0.1)

In [9]:
from agents.csv_to_graph import CsvToGraphAgent
from agents.csv_to_graph import State
from utils.duckdb_helpers import execute_sql_query
from langchain_core.messages import HumanMessage

agent = CsvToGraphAgent(llm, "t_3bb988cb3ea1487c9f8c011cf7b10860")

test_question="How does ratings for the romantic comedy movies and action movies compare?"
state = State(
    messages=[
        HumanMessage(role="user", content=test_question)
    ]
)

print(agent.parse_user_question(state))
print(agent.get_unique_nouns(state))
print(agent.get_sql_query(state))
print(agent.validate_and_fix_sql(state))
print(agent.execute_sql(state))

print(agent.choose_visualization(state))

vega_spec = agent.vega_lite_visualizer(state)
print(vega_spec)

vega_spec = agent.llm_json_fix(vega_spec)
print(str(vega_spec))


{'messages': [HumanMessage(content='How does ratings for the romantic comedy movies and action movies compare?', additional_kwargs={}, response_metadata={}, role='user')], 'parsed_question': QueryParsing(is_relevant=True, relevant_tables=[RelevantTable(table_name='t_3bb988cb3ea1487c9f8c011cf7b10860', columns=['Genre', 'IMDB Rating'], noun_columns=['Genre'])])}
{'unique_nouns': [{'Comedy', 'Western', 'Action', 'Drama', 'Musical', 'Romantic Comedy', 'Black Comedy', 'Adventure', 'Concert', 'Documentary', 'Thriller', 'Horror'}]}
{'messages': [HumanMessage(content='How does ratings for the romantic comedy movies and action movies compare?', additional_kwargs={}, response_metadata={}, role='user')], 'parsed_question': QueryParsing(is_relevant=True, relevant_tables=[RelevantTable(table_name='t_3bb988cb3ea1487c9f8c011cf7b10860', columns=['Genre', 'IMDB Rating'], noun_columns=['Genre'])]), 'unique_nouns': [{'Comedy', 'Western', 'Action', 'Drama', 'Musical', 'Romantic Comedy', 'Black Comedy', 'A


**Database Schema:**
`"Schema for database t_3bb988cb3ea1487c9f8c011cf7b10860
Title VARCHAR
Worldwide Gross BIGINT
Production Budget BIGINT
Release Year BIGINT
Content Rating VARCHAR
Running Time BIGINT
Genre VARCHAR
Creative Type VARCHAR
Rotten Tomatoes Rating BIGINT
IMDB Rating DOUBLE

Example rows:Title=From Dusk Till Dawn, Worldwide Gross=25728961, Production Budget=20000000, Release Year=1996, Content Rating=R, Running Time=107, Genre=Horror, Creative Type=Fantasy, Rotten Tomatoes Rating=63, IMDB Rating=7.1
Title=Broken Arrow, Worldwide Gross=148345997, Production Budget=65000000, Release Year=1996, Content Rating=R, Running Time=108, Genre=Action, Creative Type=Contemporary Fiction, Rotten Tomatoes Rating=55, IMDB Rating=5.8
Title=City Hall, Worldwide Gross=20278055, Production Budget=40000000, Release Year=1996, Content Rating=R, Running Time=111, Genre=Drama, Creative Type=Contemporary Fiction, Rotten Tomatoes Rating=55, IMDB Rating=6.1
"`

===Generated SQL query:
SELECT Genre

In [6]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver


# Initialize checkpointer for conversation memory
checkpointer = InMemorySaver()

# Create graph builder
graph_builder = StateGraph(State)


# Add nodes
graph_builder.add_node("parse_user_question", agent.parse_user_question)
graph_builder.add_node("get_unique_nouns", agent.get_unique_nouns)
graph_builder.add_node("get_sql_query", agent.get_sql_query)
graph_builder.add_node("validate_and_fix_sql", agent.validate_and_fix_sql)
graph_builder.add_node("execute_sql", agent.execute_sql)
graph_builder.add_node("choose_visualization", agent.choose_visualization)
graph_builder.add_node("visualize", agent.visualize)
graph_builder.add_node("answer_with_visual", agent.answer_with_visual)
graph_builder.add_node("answer_with_data", agent.answer_with_data)
##graph_builder.add_node("vega_lite_visualizer", vega_lite_visualizer)

graph_builder.add_edge(START, "parse_user_question")
graph_builder.add_conditional_edges("parse_user_question", agent.route_after_question_parse)
graph_builder.add_edge("get_unique_nouns", "get_sql_query")
graph_builder.add_edge("get_sql_query", "validate_and_fix_sql")
graph_builder.add_edge("validate_and_fix_sql", "execute_sql")
graph_builder.add_edge("execute_sql", "choose_visualization")
graph_builder.add_conditional_edges("choose_visualization", agent.route_after_choose_visualization)
graph_builder.add_edge("visualize", "answer_with_visual")

# Compile graph with checkpointer
agent_graph = graph_builder.compile(
    checkpointer=checkpointer,
)

In [7]:
import uuid
from langchain_core.messages import HumanMessage
import gradio as gr
from PIL import Image

def create_thread_id():
    """Generate a unique thread ID for each conversation"""
    return {"configurable": {"thread_id": str(uuid.uuid4())}}

def clear_session():
    """Clear thread for new conversation"""
    global config
    agent_graph.checkpointer.delete_thread(config["configurable"]["thread_id"])
    config = create_thread_id()

def chat_with_agent(message, history):
    current_state = agent_graph.get_state(config)
    
    if current_state.next:
        # Resume interrupted conversation
        agent_graph.update_state(config,{"messages": [HumanMessage(content=message.strip())]})
        result = agent_graph.invoke(None, config)
    else:
        # Start new query
        result = agent_graph.invoke({"messages": [HumanMessage(content=message.strip())]},config)
    
    image = result.get("generated_chart", None)

    print(result)
    print(image)
    
    if (image is not None):
        # Convert PNG bytes → PIL Image
        pil_image = Image.open(image)

        return [result['messages'][-1].content, gr.Image(pil_image)]

    return result['messages'][-1].content

# Initialize thread configuration
config = create_thread_id()

# Create Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        height=600,
        placeholder="<strong>Ask me anything!</strong><br><em>I'll search, reason, and act to give you the best answer :)</em>"
    )
    chatbot.clear(clear_session)
    gr.ChatInterface(fn=chat_with_agent, chatbot=chatbot)

/home/daniyusra/projects/manantara-playground/material-agents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



**Database Schema:**
`"Schema for database t_3bb988cb3ea1487c9f8c011cf7b10860
Title VARCHAR
Worldwide Gross BIGINT
Production Budget BIGINT
Release Year BIGINT
Content Rating VARCHAR
Running Time BIGINT
Genre VARCHAR
Creative Type VARCHAR
Rotten Tomatoes Rating BIGINT
IMDB Rating DOUBLE

Example rows:Title=From Dusk Till Dawn, Worldwide Gross=25728961, Production Budget=20000000, Release Year=1996, Content Rating=R, Running Time=107, Genre=Horror, Creative Type=Fantasy, Rotten Tomatoes Rating=63, IMDB Rating=7.1
Title=Broken Arrow, Worldwide Gross=148345997, Production Budget=65000000, Release Year=1996, Content Rating=R, Running Time=108, Genre=Action, Creative Type=Contemporary Fiction, Rotten Tomatoes Rating=55, IMDB Rating=5.8
Title=City Hall, Worldwide Gross=20278055, Production Budget=40000000, Release Year=1996, Content Rating=R, Running Time=111, Genre=Drama, Creative Type=Contemporary Fiction, Rotten Tomatoes Rating=55, IMDB Rating=6.1
"`

===Generated SQL query:
SELECT Genre